In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d


def extract_peaks(vec):
    """
    vec: array-like, the periodogram in original scale
    returns: array with log-peaks, otherwise 0
    """
    vec = np.asarray(vec)
    n = len(vec)

    # Rolling median (window size 9, centered)
    med_psd = (
        pd.Series(vec)
        .rolling(window=9, center=True, min_periods=1)
        .median()
        .to_numpy()
    )

    # Power ratio
    ratio_med = vec / (med_psd + np.finfo(float).eps)
    log_ratio_med = np.log(ratio_med)

    # Replace NaNs with mean
    mean_lr = np.nanmean(log_ratio_med)
    log_ratio_med = np.where(np.isnan(log_ratio_med), mean_lr, log_ratio_med)

    # Threshold = Q3 + 1.5 * (Q3 - Q1)
    Q1 = np.nanquantile(log_ratio_med, 0.25)
    Q3 = np.nanquantile(log_ratio_med, 0.75)
    threshold = Q3 + 1.5 * (Q3 - Q1)

    # Find values greater than threshold
    out = np.zeros(n)
    out[log_ratio_med > threshold] = log_ratio_med[log_ratio_med > threshold]

    return out


from scipy.signal.windows import gaussian


def smooth_peaks(vec, d):
    """
    Returns the smoothed vector with peaks.

    Parameters
    ----------
    vec : array-like, Vector with peaks.
    d : int, Propagation distance to the left and right.
    """
    vec = np.asarray(vec)
    out = vec.copy()
    n = len(vec)

    # Gaussian decay (like gsignal::gausswin in R)
    gauss_win = gaussian(2 * d, std=2.5)
    dec = gauss_win[:d][::-1]  # take first d and reverse

    for i in range(1, d + 1):
        # propagate to the left
        aux1 = np.concatenate([vec[i:], np.zeros(i)])
        # propagate to the right
        aux2 = np.concatenate([np.zeros(i), vec[: n - i]])

        # apply Gaussian decay
        aux1 = aux1 * dec[i - 1]
        aux2 = aux2 * dec[i - 1]

        # update with maximum
        out = np.maximum(out, aux1)
        out = np.maximum(out, aux2)

    # Areas between peaks: aassign
    m0 = np.sum(out) * 0.05  # 5 percent of total peaks
    n0 = np.count_nonzero(out)  # number of non-zeros
    if n0 > 0:
        min_value = m0 / n0
        out[out <= min_value] = min_value

    return out


def knotLoc(vec, k, degree, eqSpaced=False):
    """
    Compute knot locations for B-spline densities.

    Parameters
    ----------
    vec : array-like, Numeric vector containing the smoothed peaks.
    k : int,  Number of B-spline densities.
    degree : int
        Degree of the B-spline densities.
    eqSpaced : bool, optional (default=False)
        If True, returns equidistant knots in [0,1].

    Returns
    -------
    knots : ndarray
        Knot positions in [0,1].
    """
    K = k - degree + 1  # number of internal knots in [0,1]
    vec = np.asarray(vec)

    if eqSpaced:
        knots = np.linspace(0, 1, K)
        return knots

    N = len(vec)

    # Normalized density
    dens = vec / np.sum(vec)
    cumf = np.cumsum(dens)

    # Distribution function (maps [0,1] -> [0,1])
    df = interp1d(
        np.linspace(0, 1, N),
        cumf,
        kind="linear",
        bounds_error=False,
        fill_value=(0, 1),
    )

    # Inverse distribution function
    grid = np.linspace(0, 1, N)
    dfvec = df(grid)
    invDf = interp1d(
        dfvec, grid, kind="linear", bounds_error=False, fill_value=np.nan
    )

    # Internal points (exclude 0 and 1)
    v = np.linspace(0, 1, K)
    v = v[1:-1]

    knots = np.concatenate(([0], invDf(v), [1]))

    return knots

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# 1. Identifying peaks
aux = extract_peaks(lvk_data.psd)

# 2. Smoothed peaks
r = smooth_peaks(aux, 40)

# 3. Plot identified peaks
plt.figure(figsize=(8, 4))
plt.plot(lvk_data.freqs, aux, linestyle="-", color="blue")
plt.title("Identified Peaks")
plt.xlabel("Frequency")
plt.ylabel("log Peaks")
plt.show()

# 4. Plot areas to allocate knots (smoothed peaks)
plt.figure(figsize=(8, 4))
plt.plot(lvk_data.freqs, r, linestyle="-", color="red")
plt.title("Smoothed peaks")
plt.xlabel("Frequency")
plt.ylabel("Smoothed Peaks")
plt.show()

#####################
### Ploting Knots ###
#####################

# 1. Compute knots
k = knotLoc(vec=r, k=100, degree=3)
maxFreq = np.max(lvk_data.freqs)  # Maximum frequency

# 2. Plot LVK periodogram
plt.figure(figsize=(10, 4))
plt.plot(
    lvk_data.freqs, np.log(lvk_data.psd), linestyle="-", color="lightgray"
)
plt.title("LVK Periodogram")
plt.xlabel("Frequency")
plt.ylabel("Log-periodogram")

# Red points: knot locations
plt.scatter(k * maxFreq, np.full_like(k, -35), color="red", label="Knots")

# 3. Distribution function from smoothed peaks
N = len(r)
dens = r / np.sum(r)
cumf = np.cumsum(dens)

# Interpolation function (distribution function)
df = interp1d(
    np.linspace(0, 1, N),
    cumf,
    kind="linear",
    bounds_error=False,
    fill_value=(0, 1),
)

# 4. Plot areas to allocate knots
plt.figure(figsize=(10, 4))
plt.plot(
    lvk_data.freqs,
    r / np.max(r),
    linestyle="-",
    color="lightgray",
    label="Normalized Smoothed Peaks",
)

# Add cumulative distribution (red line)
plt.plot(
    lvk_data.freqs,
    df(np.array(lvk_data.freqs) / 2048),
    color="red",
    label="CDF",
)

plt.title("Smoothed peaks & CDF")
plt.xlabel("Frequency")
plt.ylabel("Normalized Peaks / CDF")
plt.legend()
plt.show()